In [76]:
import pandas as pd
import numpy as np

# ANNUAL

In [77]:
pse_annual_financials = pd.read_excel("PSE Companies Financial Reports.xlsx", sheet_name="Annual Financial Reports")

In [78]:
# Function to clean numbers: remove commas, handle parentheses as negatives, handle dashes
def clean_number(x):
    x = str(x).replace(',', '').strip()         # remove commas and spaces
    if x in ['', '-', 'nan', 'NaN']:            # treat missing or dash as 0
        return 0
    if x.startswith('(') and x.endswith(')'):   # convert (123) to -123
        x = '-' + x[1:-1]
    try:
        return round(float(x), 2)               # keep two decimal places
    except ValueError:
        return 0   # fallback if any unexpected string appears

# Apply cleaning function
pse_annual_financials['Current Year'] = pse_annual_financials['Current Year'].apply(clean_number)
pse_annual_financials['Previous Year'] = pse_annual_financials['Previous Year'].apply(clean_number)

# Create a master list of all companies with currency and fiscal period
all_companies = pse_annual_financials[['Company Name', 'Fiscal Period', 'Currency']].drop_duplicates()

# Pivot to wide format
pse_annual_financials_clean = pse_annual_financials.pivot_table(
    index=['Company Name', 'Fiscal Period', 'Currency'],
    columns='Item',
    values=['Current Year', 'Previous Year']
)

# Flatten MultiIndex columns
pse_annual_financials_clean.columns = [f'{metric}_{year}' for year, metric in pse_annual_financials_clean.columns]
pse_annual_financials_clean = pse_annual_financials_clean.reset_index()

# Merge with master list to retain all companies
pse_annual_financials_clean = all_companies.merge(
    pse_annual_financials_clean,
    on=['Company Name', 'Fiscal Period', 'Currency'],
    how='left'
)

# Replace all NaNs in numeric columns with 0
numeric_cols = [col for col in pse_annual_financials_clean.columns if col not in ['Company Name', 'Fiscal Period', 'Currency']]
pse_annual_financials_clean[numeric_cols] = pse_annual_financials_clean[numeric_cols].fillna(0)

In [79]:
# Set USD to PHP exchange rate
USD_to_PHP = 59.0  # as of 11/10/2025

# First, standardize the currency labels (for detecting scale)
def normalize_currency(x):
    if pd.isna(x):
        return 'PHP'  # default to PHP if missing
    x = str(x).lower().strip()
    if any(word in x for word in ['php', 'peso']):
        return 'PHP'
    elif any(word in x for word in ['usd', 'us$']):
        return 'USD'
    return x  # leave any other currencies as-is

# Apply normalization
pse_annual_financials_clean['Currency'] = pse_annual_financials_clean['Currency'].apply(normalize_currency)

# Store original currency string for scale detection
pse_annual_financials_clean['Currency_Original'] = pse_annual_financials_clean['Currency']

# Function to adjust for thousands/millions
def adjust_scale(orig_currency):
    orig_currency = str(orig_currency).lower()
    if any(word in orig_currency for word in ['thousand', "'000", "000"]):
        return 1_000
    elif 'million' in orig_currency:
        return 1_000_000
    else:
        return 1

# Compute scaling factor
pse_annual_financials_clean['Scale_Factor'] = pse_annual_financials_clean['Currency_Original'].apply(adjust_scale)

# Identify numeric columns
numeric_cols = [col for col in pse_annual_financials_clean.columns 
                if col not in ['Company Name', 'Fiscal Period', 'Currency', 'Currency_Original', 'Scale_Factor']]

# Apply scale factor
for col in numeric_cols:
    pse_annual_financials_clean[col] = pse_annual_financials_clean[col] * pse_annual_financials_clean['Scale_Factor']

# Convert USD rows to PHP
usd_mask = pse_annual_financials_clean['Currency'] == 'USD'
for col in numeric_cols:
    pse_annual_financials_clean.loc[usd_mask, col] *= USD_to_PHP

# After conversion, set all currency to PHP
pse_annual_financials_clean['Currency'] = 'PHP'

# Drop helper columns
pse_annual_financials_clean = pse_annual_financials_clean.drop(columns=['Currency_Original', 'Scale_Factor'])

# Set pandas display option to show numbers with commas and 2 decimals
pd.set_option('display.float_format', '{:,.2f}'.format)

# Preview final DataFrame
pse_annual_financials_clean

,Company Name,Fiscal Period,Currency,Book Value Per Share_Current Year,Current Assets_Current Year,Current Liabilities_Current Year,Earnings/(Loss) Per Share (Basic)_Current Year,Earnings/(Loss) Per Share (Diluted)_Current Year,Gross Expense_Current Year,Gross Revenue_Current Year,...,Gross Expense_Previous Year,Gross Revenue_Previous Year,Income/(Loss) Before Tax_Previous Year,Net Income/(Loss) After Tax_Previous Year,Net Income/(Loss) Attributable to Parent_Previous Year,Retained Earnings/(Deficit)_Previous Year,Stockholders' Equity_Previous Year,Stockholders' Equity - Parent_Previous Year,Total Assets_Previous Year,Total Liabilities_Previous Year
0,ASIA AMALGAMATED HOLDINGS CORPORATION,"Dec 31, 2013",PHP,-0.02,"397,025.00","23,669,384.00",0.00,0.00,"1,364,287.00",340.00,...,"3,103,046.00","5,839.00","-3,097,207.00","-3,097,207.00","-3,097,157.00","-830,092,017.00","-18,176,575.00","-18,176,425.00","3,952,649.00","22,129,224.00"
1,"ABACORE CAPITAL HOLDINGS, INC.","Dec 31, 2024",PHP,5.01,"684,939,635.00","1,525,237,968.00",0.60,0.60,"163,291,904.00","3,457,155,961.00",...,"163,694,824.00","3,028,532,061.00","2,864,837,237.00","2,435,550,871.00","2,451,443,075.00","14,090,500,587.00","19,128,699,840.00","6,352,566,583.00","23,288,638,023.00","4,159,938,183.00"
2,ASIABEST GROUP INTERNATIONAL INC.,"Dec 31, 2024",PHP,0.83,"254,977,647.00","4,781,076.00",0.00,0.00,"9,083,652.00",0.00,...,"8,901,465.00",0.00,"-2,224,923.00","-2,226,591.00",0.00,"-48,164,806.00","251,835,194.00",0.00,"256,854,596.00","5,019,402.00"
3,AYALA CORPORATION,"Dec 31, 2024",PHP,652.22,"584,562,518.00","389,290,509.00",64.77,64.69,"289,776,138.00","325,743,960.00",...,"262,478,576.00","289,904,747.00","64,756,411.00","55,176,358.00","38,073,084.00","297,882,907.00","674,941,361.00","396,538,552.00","1,608,713,290.00","933,771,929.00"
4,ACEN CORPORATION,"Dec 31, 2024",PHP,3.72,"59,132,109.00","29,670,253.00",0.19,0.19,"34,341,343.00","37,300,402.00",...,"39,200,966.00","36,499,133.00","9,388,972.00","9,106,674.00","7,396,140.00","24,871,807.00","173,375,857.00","143,472,317.00","284,933,500.00","111,557,643.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,VITARICH CORPORATION,"Dec 31, 2024",PHP,0.74,"3,206,286,572.00","3,128,802,760.00",0.07,0.07,"12,141,471,258.00","12,539,289,231.00",...,"12,485,252,234.00","12,495,914,596.00","6,580,697.00","13,304,916.00","13,304,916.00","303,502,861.00","1,901,342,078.00","1,901,342,078.00","5,222,496,068.00","3,321,153,990.00"
103,"VICTORIAS MILLING COMPANY, INC.","Aug 31, 2024",PHP,4.51,"5,389,875.00","1,227,708.00",0.57,0.57,"10,204,703.00","11,377,071.00",...,"13,219,888.00","15,546,991.00","1,723,593.00","1,570,672.00","1,571,839.00","8,354,491.00","11,180,308.00","11,178,292.00","13,543,683.00","2,363,375.00"
104,VIVANT CORPORATION,"Dec 31, 2024",PHP,19.60,"8,195,538,164.00","3,413,758,867.00",2.30,0.00,"8,924,332,459.00","12,201,703,051.00",...,"2,724,883,383.00","8,264,817,155.00","2,578,413,102.00","2,405,542,792.00","2,295,804,486.00","17,444,734,635.00","19,649,660,001.00","18,292,285,621.00","30,038,390,747.00","10,388,730,746.00"
105,"WELLEX INDUSTRIES, INCORPORATED","Dec 31, 2024",PHP,0.43,"60,349,587.00","14,234,212.00",0.06,0.06,"32,253,681.00","32,563,156.00",...,"29,727,138.00","25,749,089.00","8,189,583.00","3,605,543.00","3,605,543.00","-2,096,773,807.00","1,203,754,631.00","1,203,754,631.00","1,623,616,657.00","419,862,026.00"


In [80]:
# Copy the cleaned wide-format data to work on
df = pse_annual_financials_clean.copy()

# Define numeric columns we will need
curr_assets_curr = 'Current Assets_Current Year'
curr_liab_curr = 'Current Liabilities_Current Year'
tot_assets_curr = 'Total Assets_Current Year'
tot_liab_curr = 'Total Liabilities_Current Year'
equity_curr = "Stockholders' Equity_Current Year"
equity_parent_curr = "Stockholders' Equity - Parent_Current Year"
net_income_curr = 'Net Income/(Loss) After Tax_Current Year'
gross_revenue_curr = 'Gross Revenue_Current Year'
gross_expense_curr = 'Gross Expense_Current Year'
eps_basic_curr = 'Earnings/(Loss) Per Share (Basic)_Current Year'
eps_diluted_curr = 'Earnings/(Loss) Per Share (Diluted)_Current Year'
bvps_curr = 'Book Value Per Share_Current Year'

curr_assets_prev = 'Current Assets_Previous Year'
curr_liab_prev = 'Current Liabilities_Previous Year'
tot_assets_prev = 'Total Assets_Previous Year'
tot_liab_prev = 'Total Liabilities_Previous Year'
equity_prev = "Stockholders' Equity_Previous Year"
equity_parent_prev = "Stockholders' Equity - Parent_Previous Year"
net_income_prev = 'Net Income/(Loss) After Tax_Previous Year'
gross_revenue_prev = 'Gross Revenue_Previous Year'
gross_expense_prev = 'Gross Expense_Previous Year'
eps_basic_prev = 'Earnings/(Loss) Per Share (Basic)_Previous Year'
eps_diluted_prev = 'Earnings/(Loss) Per Share (Diluted)_Previous Year'
bvps_prev = 'Book Value Per Share_Previous Year'

# Initialize a dictionary to store metrics
metrics = {}

# Loop through current and previous year
for suffix, assets, liab, tot_assets, tot_liab, equity, equity_parent, net_income, revenue, gross_expense, eps_b, eps_d, bvps in [
    ('Current Year', curr_assets_curr, curr_liab_curr, tot_assets_curr, tot_liab_curr, equity_curr, equity_parent_curr, net_income_curr, gross_revenue_curr, gross_expense_curr, eps_basic_curr, eps_diluted_curr, bvps_curr),
    ('Previous Year', curr_assets_prev, curr_liab_prev, tot_assets_prev, tot_liab_prev, equity_prev, equity_parent_prev, net_income_prev, gross_revenue_prev, gross_expense_prev, eps_basic_prev, eps_diluted_prev, bvps_prev)
]:
    # Liquidity ratios
    metrics[f'Current Ratio_{suffix}'] = df[assets] / df[liab]
    
    # Leverage / solvency
    metrics[f'Debt to Equity_{suffix}'] = df[tot_liab] / df[equity]
    metrics[f'Debt Ratio_{suffix}'] = df[tot_liab] / df[tot_assets]
    metrics[f'Equity Ratio_{suffix}'] = df[equity] / df[tot_assets]
    
    # Profitability
    metrics[f'ROA_{suffix}'] = df[net_income] / df[tot_assets]
    metrics[f'ROE_{suffix}'] = df[net_income] / df[equity]
    metrics[f'Profit Margin_{suffix}'] = df[net_income] / df[revenue]
    metrics[f'Gross Margin_{suffix}'] = (df[revenue] - df[gross_expense]) / df[revenue]
    
    # Efficiency
    metrics[f'Asset Turnover_{suffix}'] = df[revenue] / df[tot_assets]
    
    # Market metrics
    metrics[f'EPS (Basic)_{suffix}'] = df[eps_b]
    metrics[f'EPS (Diluted)_{suffix}'] = df[eps_d]
    metrics[f'Book Value Per Share_{suffix}'] = df[bvps]
    
# Combine into a new DataFrame
df_annual_metrics = pd.DataFrame(metrics)
df_annual_metrics['Company Name'] = df['Company Name']
df_annual_metrics['Fiscal Period'] = df['Fiscal Period']

# Reorder columns so identifiers are first
cols = ['Company Name', 'Fiscal Period'] + [c for c in df_annual_metrics.columns if c not in ['Company Name', 'Fiscal Period']]
df_annual_metrics = df_annual_metrics[cols]

# Replace any infinite or NaN ratios with 0
df_annual_metrics.replace([np.inf, -np.inf], np.nan, inplace=True)
df_annual_metrics.fillna(0, inplace=True)

# Display
df_annual_metrics

,Company Name,Fiscal Period,Current Ratio_Current Year,Debt to Equity_Current Year,Debt Ratio_Current Year,Equity Ratio_Current Year,ROA_Current Year,ROE_Current Year,Profit Margin_Current Year,Gross Margin_Current Year,...,Debt Ratio_Previous Year,Equity Ratio_Previous Year,ROA_Previous Year,ROE_Previous Year,Profit Margin_Previous Year,Gross Margin_Previous Year,Asset Turnover_Previous Year,EPS (Basic)_Previous Year,EPS (Diluted)_Previous Year,Book Value Per Share_Previous Year
0,ASIA AMALGAMATED HOLDINGS CORPORATION,"Dec 31, 2013",0.02,-1.21,5.73,-4.73,-0.33,0.07,"-4,011.61","-4,011.61",...,5.60,-4.60,-0.78,0.17,-530.43,-530.43,0.00,0.00,0.00,-0.02
1,"ABACORE CAPITAL HOLDINGS, INC.","Dec 31, 2024",0.45,0.22,0.18,0.82,0.10,0.12,0.76,0.95,...,0.18,0.82,0.10,0.13,0.80,0.95,0.13,0.56,0.56,4.36
2,ASIABEST GROUP INTERNATIONAL INC.,"Dec 31, 2024",53.33,0.02,0.02,0.98,-0.01,-0.01,0.00,0.00,...,0.02,0.98,-0.01,-0.01,0.00,0.00,0.00,0.00,0.00,0.84
3,AYALA CORPORATION,"Dec 31, 2024",1.50,1.41,0.59,0.41,0.04,0.08,0.19,0.11,...,0.58,0.42,0.03,0.08,0.19,0.09,0.18,58.75,58.64,598.15
4,ACEN CORPORATION,"Dec 31, 2024",1.99,1.09,0.52,0.48,0.03,0.06,0.27,0.08,...,0.39,0.61,0.03,0.05,0.25,-0.07,0.13,0.17,0.17,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,VITARICH CORPORATION,"Dec 31, 2024",1.02,1.58,0.61,0.39,0.04,0.10,0.02,0.03,...,0.64,0.36,0.00,0.01,0.00,0.00,2.39,0.00,0.00,0.62
103,"VICTORIAS MILLING COMPANY, INC.","Aug 31, 2024",4.39,0.16,0.14,0.86,0.11,0.13,0.14,0.10,...,0.17,0.83,0.12,0.14,0.10,0.15,1.15,0.57,0.57,4.08
104,VIVANT CORPORATION,"Dec 31, 2024",2.40,0.49,0.33,0.67,0.08,0.12,0.22,0.27,...,0.35,0.65,0.08,0.12,0.29,0.67,0.28,2.24,0.00,17.87
105,"WELLEX INDUSTRIES, INCORPORATED","Dec 31, 2024",4.24,0.30,0.23,0.77,0.11,0.14,6.11,0.01,...,0.26,0.74,0.00,0.00,0.14,-0.15,0.02,0.00,0.00,0.37


# QUARTERLY

In [81]:
pse_quarterly_financials = pd.read_excel("PSE Companies Financial Reports.xlsx", sheet_name="Quarterly Financial Reports")

In [82]:
# Set display format
pd.set_option('display.float_format', '{:,.2f}'.format)

# Step 1: Clean numeric columns
def clean_number(x):
    x = str(x).replace(',', '').replace(' ', '').strip()
    if x in ['', '-', 'nan', 'NaN']:
        return 0.0
    if x.startswith('(') and x.endswith(')'):
        x = '-' + x[1:-1]
    try:
        return round(float(x), 2)
    except ValueError:
        return 0.0

numeric_cols = ['Current Period', 'Previous Period', 'Current 3 Months', 
                'Previous 3 Months', 'Current YTD', 'Previous YTD']

for col in numeric_cols:
    pse_quarterly_financials[col] = pse_quarterly_financials[col].apply(clean_number)

# Step 2: Fill missing currencies with PHP
pse_quarterly_financials['Currency'] = pse_quarterly_financials['Currency'].fillna('PHP')

# Step 3: Create master list of all companies + fiscal periods + currencies
all_companies = pse_quarterly_financials[['Company Name', 'Fiscal Period', 'Currency']].drop_duplicates()

# Step 4: Pivot to wide format
pse_quarterly_financials_clean = pse_quarterly_financials.pivot_table(
    index=['Company Name', 'Fiscal Period', 'Currency'],
    columns='Item',
    values=numeric_cols
)

# Flatten MultiIndex columns
pse_quarterly_financials_clean.columns = [f'{metric}_{period}' for period, metric in pse_quarterly_financials_clean.columns]
pse_quarterly_financials_clean = pse_quarterly_financials_clean.reset_index()

# Step 5: Merge with master list to retain all companies
pse_quarterly_financials_clean = all_companies.merge(
    pse_quarterly_financials_clean,
    on=['Company Name', 'Fiscal Period', 'Currency'],
    how='left'
)

# Step 6: Replace all NaNs in numeric columns with 0
numeric_cols_cleaned = [col for col in pse_quarterly_financials_clean.columns 
                        if col not in ['Company Name', 'Fiscal Period', 'Currency']]
pse_quarterly_financials_clean[numeric_cols_cleaned] = pse_quarterly_financials_clean[numeric_cols_cleaned].fillna(0)

In [83]:
# Set USD to PHP exchange rate
USD_to_PHP = 59.0  # as of 11/10/2025

# Standardize currency labels
def normalize_currency(x):
    if pd.isna(x):
        return 'PHP'
    x = str(x).lower().strip()
    if any(word in x for word in ['php', 'peso']):
        return 'PHP'
    elif any(word in x for word in ['usd', 'us$']):
        return 'USD'
    return x  # leave any other currencies as-is

# Apply normalization
pse_quarterly_financials_clean['Currency'] = pse_quarterly_financials_clean['Currency'].apply(normalize_currency)

# Store original currency string for scale detection
pse_quarterly_financials_clean['Currency_Original'] = pse_quarterly_financials_clean['Currency']

# Function to adjust for thousands/millions
def adjust_scale(orig_currency):
    orig_currency = str(orig_currency).lower()
    if any(word in orig_currency for word in ['thousand', "'000", "000"]):
        return 1_000
    elif 'million' in orig_currency:
        return 1_000_000
    else:
        return 1

# Compute scaling factor
pse_quarterly_financials_clean['Scale_Factor'] = pse_quarterly_financials_clean['Currency_Original'].apply(adjust_scale)

# Identify numeric columns
numeric_cols = [col for col in pse_quarterly_financials_clean.columns 
                if col not in ['Company Name', 'Fiscal Period', 'Currency', 'Currency_Original', 'Scale_Factor']]

# Apply scale factor
for col in numeric_cols:
    pse_quarterly_financials_clean[col] = pse_quarterly_financials_clean[col] * pse_quarterly_financials_clean['Scale_Factor']

# Convert USD rows to PHP
usd_mask = pse_quarterly_financials_clean['Currency'] == 'USD'
for col in numeric_cols:
    pse_quarterly_financials_clean.loc[usd_mask, col] *= USD_to_PHP

# After conversion, set all currency to PHP
pse_quarterly_financials_clean['Currency'] = 'PHP'

# Drop helper columns
pse_quarterly_financials_clean = pse_quarterly_financials_clean.drop(columns=['Currency_Original', 'Scale_Factor'])

# Set pandas display option for commas and 2 decimals
pd.set_option('display.float_format', '{:,.2f}'.format)

# Preview final DataFrame
pse_quarterly_financials_clean

,Company Name,Fiscal Period,Currency,Book Value Per Share_Current 3 Months,Current Assets_Current 3 Months,Current Liabilities_Current 3 Months,Earnings/(Loss) Per Share (Basic)_Current 3 Months,Earnings/(Loss) Per Share (Diluted)_Current 3 Months,Gross Expense_Current 3 Months,Gross Revenue_Current 3 Months,...,Gross Expense_Previous YTD,Gross Revenue_Previous YTD,Income/(Loss) Before Tax_Previous YTD,Net Income/(Loss) After Tax_Previous YTD,Net Income/(Loss) Attributable to Parent_Previous YTD,Retained Earnings/(Deficit)_Previous YTD,Stockholders' Equity_Previous YTD,Stockholders' Equity - Parent_Previous YTD,Total Assets_Previous YTD,Total Liabilities_Previous YTD
0,ASIA AMALGAMATED HOLDINGS CORPORATION,"Jun 30, 2014",PHP,0.00,0.00,0.00,0.00,0.00,"124,000.00",80.00,...,"1,000,036.00",180.00,"-999,856.00","-999,856.00","-999,856.00",0.00,0.00,0.00,0.00,0.00
1,"ABACORE CAPITAL HOLDINGS, INC.","Jun 30, 2025",PHP,0.00,0.00,0.00,-0.01,0.00,"29,480,769.00","2,868,846.00",...,"63,104,526.00","132,933,736.00","69,829,210.00","69,829,210.00","75,704,268.00",0.00,0.00,0.00,0.00,0.00
2,ASIABEST GROUP INTERNATIONAL INC.,"Jun 30, 2025",PHP,0.00,0.00,0.00,0.00,0.00,"2,200,921.00",0.00,...,"4,201,835.00",0.00,"-311,017.00","-311,017.00",0.00,0.00,0.00,0.00,0.00,0.00
3,AYALA CORPORATION,"Jun 30, 2025",PHP,0.00,0.00,0.00,16.57,16.54,"68,087,263.00","79,390,091.00",...,"141,900,949.00","156,639,291.00","37,764,916.00","32,511,055.00","22,286,487.00",0.00,0.00,0.00,0.00,0.00
4,ACEN CORPORATION,"Sep 30, 2025",PHP,0.00,0.00,0.00,0.01,0.01,"7,224,006.00","7,273,027.00",...,"24,869,559.00","28,084,895.00","9,569,859.00","8,966,449.00","8,144,445.00",0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,VITARICH CORPORATION,"Jun 30, 2025",PHP,0.00,0.00,0.00,0.00,0.00,"2,909,660,703.00","2,924,326,680.00",...,"5,846,640,614.00","6,089,373,398.00","205,059,044.00","167,035,481.00","167,035,481.00",0.00,0.00,0.00,0.00,0.00
103,"VICTORIAS MILLING COMPANY, INC.","May 31, 2025",PHP,0.00,0.00,0.00,0.07,0.07,"3,836,462.00","4,178,997.00",...,"9,063,591.00","10,048,794.00","1,298,097.00","1,220,375.00","1,219,984.00",0.00,0.00,0.00,0.00,0.00
104,VIVANT CORPORATION,"Jun 30, 2025",PHP,0.00,0.00,0.00,0.66,0.00,"2,179,520,917.00","3,017,650,021.00",...,"4,487,886,711.00","5,557,217,453.00","932,650,401.00","871,885,989.00","877,386,235.00",0.00,0.00,0.00,0.00,0.00
105,"WELLEX INDUSTRIES, INCORPORATED","Sep 30, 2025",PHP,0.00,0.00,0.00,0.00,0.00,"6,569,554.00","4,998,937.00",...,"24,638,201.00","24,471,671.00","-518,999.00","-518,999.00","-518,999.00",0.00,0.00,0.00,0.00,0.00


In [84]:
# Copy cleaned quarterly data
df = pse_quarterly_financials_clean.copy()

# Identify columns by suffix
suffixes = ['_Current 3 Months', '_Current Period', '_Current YTD',
            '_Previous 3 Months', '_Previous Period', '_Previous YTD']

# List of numeric columns
numeric_cols = [col for col in df.columns if any(col.endswith(s) for s in suffixes)]

# Initialize master metrics DataFrame with Fiscal Period and Currency
companies_info = df.groupby('Company Name').agg({
    'Fiscal Period': lambda x: x.dropna().max(),
    'Currency': lambda x: x.dropna().max()
}).reset_index()

df_quarterly_metrics = companies_info.copy()

# Define categories of financial items
balance_items = [
    'Total Assets', 'Total Liabilities', 'Current Assets', 'Current Liabilities',
    'Stockholders\' Equity', 'Stockholders\' Equity - Parent', 'Retained Earnings/(Deficit)'
]

income_items = [
    'Gross Revenue', 'Gross Expense', 'Income/(Loss) Before Tax', 'Net Income/(Loss) After Tax',
    'Net Income/(Loss) Attributable to Parent'
]

eps_items = [
    'Earnings/(Loss) Per Share (Basic)', 'Earnings/(Loss) Per Share (Diluted)', 'Book Value Per Share'
]

# Aggregate Current and Previous values per company
for company in df_quarterly_metrics['Company Name']:
    temp = df[df['Company Name'] == company]
    
    # Balance sheet: latest values
    for item in balance_items:
        current_cols = [col for col in temp.columns if col.startswith(item) and 'Current' in col]
        previous_cols = [col for col in temp.columns if col.startswith(item) and 'Previous' in col]
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Current'] = temp[current_cols].max().max()
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Previous'] = temp[previous_cols].max().max()
    
    # Income statement: sum across periods
    for item in income_items:
        current_cols = [col for col in temp.columns if col.startswith(item) and 'Current' in col]
        previous_cols = [col for col in temp.columns if col.startswith(item) and 'Previous' in col]
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Current'] = temp[current_cols].sum().sum()
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Previous'] = temp[previous_cols].sum().sum()
    
    # EPS and Book Value Per Share: latest period
    for item in eps_items:
        current_cols = [col for col in temp.columns if col.startswith(item) and 'Current' in col]
        previous_cols = [col for col in temp.columns if col.startswith(item) and 'Previous' in col]
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Current'] = temp[current_cols].max().max()
        df_quarterly_metrics.loc[df_quarterly_metrics['Company Name'] == company, item + '_Previous'] = temp[previous_cols].max().max()

# --- DERIVED METRICS ---

# Liquidity ratios
df_quarterly_metrics['Current_Ratio_Current'] = df_quarterly_metrics['Current Assets_Current'] / df_quarterly_metrics['Current Liabilities_Current']
df_quarterly_metrics['Current_Ratio_Previous'] = df_quarterly_metrics['Current Assets_Previous'] / df_quarterly_metrics['Current Liabilities_Previous']

df_quarterly_metrics['Quick_Ratio_Current'] = df_quarterly_metrics['Current Assets_Current'] / df_quarterly_metrics['Current Liabilities_Current']
df_quarterly_metrics['Quick_Ratio_Previous'] = df_quarterly_metrics['Current Assets_Previous'] / df_quarterly_metrics['Current Liabilities_Previous']

# Leverage ratios
df_quarterly_metrics['Debt_to_Equity_Current'] = df_quarterly_metrics['Total Liabilities_Current'] / df_quarterly_metrics['Stockholders\' Equity_Current']
df_quarterly_metrics['Debt_to_Equity_Previous'] = df_quarterly_metrics['Total Liabilities_Previous'] / df_quarterly_metrics['Stockholders\' Equity_Previous']
df_quarterly_metrics['Debt_Ratio_Current'] = df_quarterly_metrics['Total Liabilities_Current'] / df_quarterly_metrics['Total Assets_Current']
df_quarterly_metrics['Debt_Ratio_Previous'] = df_quarterly_metrics['Total Liabilities_Previous'] / df_quarterly_metrics['Total Assets_Previous']
df_quarterly_metrics['Equity_Ratio_Current'] = df_quarterly_metrics['Stockholders\' Equity_Current'] / df_quarterly_metrics['Total Assets_Current']
df_quarterly_metrics['Equity_Ratio_Previous'] = df_quarterly_metrics['Stockholders\' Equity_Previous'] / df_quarterly_metrics['Total Assets_Previous']

# Profitability ratios
df_quarterly_metrics['Net_Profit_Margin_Current'] = df_quarterly_metrics['Net Income/(Loss) After Tax_Current'] / df_quarterly_metrics['Gross Revenue_Current']
df_quarterly_metrics['Net_Profit_Margin_Previous'] = df_quarterly_metrics['Net Income/(Loss) After Tax_Previous'] / df_quarterly_metrics['Gross Revenue_Previous']
df_quarterly_metrics['ROA_Current'] = df_quarterly_metrics['Net Income/(Loss) Attributable to Parent_Current'] / df_quarterly_metrics['Total Assets_Current']
df_quarterly_metrics['ROA_Previous'] = df_quarterly_metrics['Net Income/(Loss) Attributable to Parent_Previous'] / df_quarterly_metrics['Total Assets_Previous']
df_quarterly_metrics['ROE_Current'] = df_quarterly_metrics['Net Income/(Loss) Attributable to Parent_Current'] / df_quarterly_metrics['Stockholders\' Equity - Parent_Current']
df_quarterly_metrics['ROE_Previous'] = df_quarterly_metrics['Net Income/(Loss) Attributable to Parent_Previous'] / df_quarterly_metrics['Stockholders\' Equity - Parent_Previous']

# Efficiency ratios
df_quarterly_metrics['Asset_Turnover_Current'] = df_quarterly_metrics['Gross Revenue_Current'] / df_quarterly_metrics['Total Assets_Current']
df_quarterly_metrics['Asset_Turnover_Previous'] = df_quarterly_metrics['Gross Revenue_Previous'] / df_quarterly_metrics['Total Assets_Previous']

# Growth metrics
df_quarterly_metrics['Revenue_Growth'] = (df_quarterly_metrics['Gross Revenue_Current'] - df_quarterly_metrics['Gross Revenue_Previous']) / df_quarterly_metrics['Gross Revenue_Previous']
df_quarterly_metrics['Net_Income_Growth'] = (df_quarterly_metrics['Net Income/(Loss) After Tax_Current'] - df_quarterly_metrics['Net Income/(Loss) After Tax_Previous']) / df_quarterly_metrics['Net Income/(Loss) After Tax_Previous']
df_quarterly_metrics['EPS_Growth'] = (df_quarterly_metrics['Earnings/(Loss) Per Share (Basic)_Current'] - df_quarterly_metrics['Earnings/(Loss) Per Share (Basic)_Previous']) / df_quarterly_metrics['Earnings/(Loss) Per Share (Basic)_Previous']
df_quarterly_metrics['Book_Value_Growth'] = (df_quarterly_metrics['Book Value Per Share_Current'] - df_quarterly_metrics['Book Value Per Share_Previous']) / df_quarterly_metrics['Book Value Per Share_Previous']

# Fill NaNs from division by zero
df_quarterly_metrics = df_quarterly_metrics.fillna(0)

# Display all numbers with 2 decimals and commas
pd.set_option('display.float_format', '{:,.2f}'.format)

df_quarterly_metrics

,Company Name,Fiscal Period,Currency,Total Assets_Current,Total Assets_Previous,Total Liabilities_Current,Total Liabilities_Previous,Current Assets_Current,Current Assets_Previous,Current Liabilities_Current,...,ROA_Current,ROA_Previous,ROE_Current,ROE_Previous,Asset_Turnover_Current,Asset_Turnover_Previous,Revenue_Growth,Net_Income_Growth,EPS_Growth,Book_Value_Growth
0,A. SORIANO CORPORATION,"Jun 30, 2025",PHP,"32,098,576.00","32,309,106.00","3,561,798.00","4,370,453.00","23,823,217.00","23,823,217.00","2,721,553.00",...,0.09,0.14,0.10,0.17,0.39,0.42,-0.07,-0.39,-0.55,0.02
1,"ABACORE CAPITAL HOLDINGS, INC.","Jun 30, 2025",PHP,"26,253,960,539.00","26,626,080,811.00","4,465,099,408.00","4,768,900,102.00","684,939,635.00","684,939,635.00","1,525,237,968.00",...,0.00,0.01,0.01,0.03,0.01,0.01,-0.42,-0.65,0.00,0.01
2,"ABOITIZ EQUITY VENTURES, INC.","Jun 30, 2025",PHP,"942,187,856.00","893,700,554.00","554,018,599.00","497,334,978.00","212,499,004.00","212,499,004.00","205,962,093.00",...,0.01,0.02,0.05,0.06,0.23,0.26,-0.06,-0.23,-0.26,-0.00
3,ABOITIZ POWER CORPORATION,"Jun 30, 2025",PHP,"559,928,435.00","517,616,067.00","354,757,382.00","301,719,385.00","123,026,353.00","123,026,353.00","140,010,737.00",...,0.04,0.05,0.11,0.13,0.25,0.30,-0.10,-0.21,-0.26,-0.04
4,ACEN CORPORATION,"Sep 30, 2025",PHP,"343,115,295.00","329,542,231.00","185,060,526.00","172,099,031.00","59,132,109.00","59,132,109.00","29,670,253.00",...,0.01,0.03,0.02,0.07,0.09,0.11,-0.18,-0.69,-0.94,-0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,"VICTORIAS MILLING COMPANY, INC.","May 31, 2025",PHP,"15,947,641.00","14,404,533.00","2,644,668.00","2,033,753.00","6,575,943.00","5,389,875.00","1,857,562.00",...,0.10,0.12,0.12,0.14,0.95,0.92,0.13,-0.08,-0.49,-0.46
103,VITARICH CORPORATION,"Jun 30, 2025",PHP,"5,632,621,385.00","5,783,607,308.00","3,152,671,872.00","3,538,652,338.00","3,206,286,572.00","3,206,286,572.00","3,128,802,760.00",...,0.04,0.06,0.09,0.15,1.61,1.60,-0.02,-0.32,0.40,0.09
104,VIVANT CORPORATION,"Jun 30, 2025",PHP,"32,039,493,482.00","31,966,645,403.00","10,045,334,574.00","10,446,497,024.00","8,752,441,327.00","8,195,538,164.00","4,328,214,478.00",...,0.05,0.05,0.08,0.08,0.26,0.29,-0.08,0.15,0.09,0.00
105,"WELLEX INDUSTRIES, INCORPORATED","Sep 30, 2025",PHP,"1,819,795,963.00","1,820,332,744.00","422,711,675.00","418,132,170.00","62,575,938.00","60,349,587.00","14,234,212.00",...,-0.00,-0.00,-0.00,-0.00,0.01,0.02,-0.23,158.02,0.00,0.00
